#### LeetCode - SQL - #570

Refer [570. Managers with at Least 5 Direct Reports](https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/)

Write a solution to find managers with at least  **five direct reports**.

Return the result table in  **any order**.

In [0]:
data_employee = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
columns_employee = ['id', 'name', 'department', 'managerId']
schema_employee = {'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'}

In [0]:
import pandas

pandas_dataframe_employee = pandas.DataFrame(data=data_employee, columns=columns_employee).astype(schema_employee)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
dataframe_employee = spark_context.createDataFrame(pandas_dataframe_employee)
dataframe_employee.printSchema()
dataframe_employee.show(5)

In [0]:
from pyspark.sql import functions as F

# Create alias
employee = dataframe_employee.alias("employee")
manager = dataframe_employee.alias("manager")

# Join
dataframe_manager_summary \
    = employee.join(manager, F.col("employee.managerId") == F.col("manager.id"), "inner") \
              .select(F.col("manager.id").alias("manager_id"), F.col("manager.name").alias("manager_name"),F.col("employee.id").alias("employee_id"))
dataframe_manager_summary.show()

# Group and count aggregation
dataframe_manager_grouped = dataframe_manager_summary.groupBy(dataframe_manager_summary.manager_name).agg(
    F.count("*").alias("employee_count")
)
dataframe_manager_grouped.show()

# Filter
dataframe_manager_grouped.filter(dataframe_manager_grouped.employee_count >= 5).select(dataframe_manager_grouped.manager_name.alias("name")).show()